In [1]:
# Import findspark and initialize. 
import findspark
from pyspark.sql.functions import year
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

24/06/23 16:46:02 WARN Utils: Your hostname, Michaels-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.26 instead (on interface en0)
24/06/23 16:46:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/23 16:46:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/23 16:46:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df=spark.read.csv(SparkFiles.get("home_sales_revised.csv"),sep=",",header=True)
df.show(3)


24/06/23 16:46:13 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
only showing top 3 rows



In [4]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales_view")

# Now you can use SQL queries against this view
spark.sql("SELECT * FROM home_sales_view LIMIT 5").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
# Assuming 'date' is a column of type date or timestamp
#conerting date col into date type if necessary
df = df.withColumn("date",df["date"].cast("date"))
#extract year from "date" column
df=df.withColumn("year_sold",year(df["date"]))
#filter 4 bedrooms
df_four_bedrooms =df.filter(df["bedrooms"]==4)
#create a temp view of of the 4 bedrooms dataframe
df_four_bedrooms.createOrReplaceTempView("home_sales_view")
query = """
SELECT year_sold, ROUND(AVG(price), 2) AS average_price
FROM home_sales_view
WHERE bedrooms = 4
GROUP BY year_sold
ORDER BY year_sold
"""

# Execute the query and show results
result = spark.sql(query)
result.show()




+---------+-------------+
|year_sold|average_price|
+---------+-------------+
|     2019|     300263.7|
|     2020|    298353.78|
|     2021|    301819.44|
|     2022|    296363.88|
+---------+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
# Create or replace the temporary view with the DataFrame
df.createOrReplaceTempView("home_sales_view")
# SQL query to find the average price of homes each year they were built, with 3 bedrooms and 3 bathrooms
query = """
SELECT date_built AS year_built, ROUND(AVG(price), 2) AS average_price
FROM home_sales_view
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query and show results
result = spark.sql(query)
result.show()

+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

df.createOrReplaceTempView("home_sales_view")

# SQL query to find the average price of homes each year they were built, with 3 bedrooms, 3 bathrooms, two floors, and at least 2000 sqft
query = """
SELECT date_built AS year_built, ROUND(AVG(price), 2) AS average_price
FROM home_sales_view
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query and show results
result = spark.sql(query)
result.show()

+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
from pyspark.sql import functions as F

# First, ensure that your DataFrame has a view as a temporary table in the Spark session
df.createOrReplaceTempView("home_sales_view")

# SQL query to find the average price of homes per "view" rating with conditions
query = """
SELECT view, ROUND(AVG(price), 2) AS average_price
FROM home_sales_view
GROUP BY view
HAVING average_price >= 350000
ORDER BY view DESC
"""

# Import time to measure the execution time
import time
start_time = time.time()

# Execute the query
result = spark.sql(query)

# Show the results
result.show()

# Calculate and print the execution time
print(f"Query run time: {time.time() - start_time} seconds")


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

Query run time: 0.35408902168273926 seconds
--- 0.35414719581604004 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
# Register the DataFrame as a temporary view if it hasn't been registered yet
df.createOrReplaceTempView("home_sales")
df.cache()

DataFrame[id: string, date: date, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string, year_sold: int]

In [10]:
# 8. Check if the table is cached.
is_cached =spark.catalog.isCached('home_sales')
print("Is 'home_sales'cached?", is_cached)

Is 'home_sales'cached? True


In [11]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
query = """
SELECT view, ROUND(AVG(price), 2) AS average_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# Measuring the execution time of the query using the cached data
import time
start_time = time.time()
result = spark.sql(query)
result.show()
end_time = time.time()

# Print runtime
print("Query runtime with cached data: {:.2f} seconds".format(end_time - start_time))


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

Query runtime with cached data: 0.74 seconds
--- 0.735684871673584 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
# 'df' is your DataFrame containing the home sales data
df.write.partitionBy("date_built").format("parquet").mode("overwrite").save("./Starter_Code")


In [13]:
# 11. Read the formatted parquet data.
df = spark.read.parquet("../Home_Sales/Starter_Code")

# Show the first few rows of the DataFrame to verify it has been loaded correctly
df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year_sold|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|     2021|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|     2020|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|     2019|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|     2019|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|     2021|      2015|


In [14]:
# 12. Create a temporary table for the parquet data.
df_parquet = spark.read.format("parquet").load("../Home_Sales/Starter_Code")

df_parquet.createOrReplaceTempView("home_sales_parquet")

# Now you can use SQL queries to analyze the data
# For example, to view some data from the table
spark.sql("SELECT * FROM home_sales_parquet LIMIT 5").show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year_sold|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|     2021|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|     2020|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|     2019|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|     2019|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|     2021|      2015|


In [15]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.
from time import time

# Start time
start_time = time()
# Create Spark session
spark = SparkSession.builder.appName("Home Sales Analysis").getOrCreate()

# Read the partitioned Parquet data into a DataFrame
df_parquet = spark.read.format("parquet").load("../Home_Sales/Starter_Code")

# Create a temporary view
df_parquet.createOrReplaceTempView("home_sales_parquet")

# SQL query to calculate the average price per "view" rating with certain conditions
query = """
SELECT view, ROUND(AVG(price), 2) AS average_price
FROM home_sales_parquet
WHERE price >= 350000
GROUP BY view
HAVING average_price >= 350000
ORDER BY view DESC
"""



# Execute the query
result = spark.sql(query)
result.show()

# End time
end_time = time()

# Print runtime
print(f"Query runtime: {end_time - start_time:.2f} seconds")


print("--- %s seconds ---" % (time() - start_time))

24/06/23 16:48:42 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|   9|    401393.34|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

Query runtime: 0.42 seconds
--- 0.4211289882659912 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")
print("Table 'home_sales' has been uncached.")

Table 'home_sales' has been uncached.


In [17]:
# 15. Check if the home_sales is no longer cached

# Check if the 'home_sales' table is still cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' still cached? {is_cached}")


Is 'home_sales' still cached? False
